<a href="https://colab.research.google.com/github/tking20/march-madness-model/blob/main/2024_March_Madness_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Library Imports & Google Drive set-up

The data sets used for this project come from www.kaggle.com

Their yearly "March Mania" competitions provide updated data sets at no cost

In [ ]:
import pandas as pd
import csv
from google.colab import drive
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn import svm
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

drive.mount('/content/gdrive')



start_year = 1985
end_year = 2024
max_team = 3479

file_path = '/content/gdrive/MyDrive/March Madness 2024/'

#ELO Model

##For a given game, update the ELO ratings of the two schools

In [ ]:
def process_elos(df_games, elo, i):
  team1 = df_games.loc[i, 'WTeamID']
  team2 = df_games.loc[i, 'LTeamID']

  w_elo = elo[team1-3101] + 0
  l_elo = elo[team2-3101] + 0

  #adjust for home-court advantage, if not at neutral site
  if df_games.loc[i, 'WLoc'] == 'H':
    w_elo = w_elo+HCA
  if df_games.loc[i, 'WLoc'] == 'A':
    l_elo = l_elo+HCA

  mov = df_games.loc[i, 'WScore'] - df_games.loc[i, 'LScore']
  k = multiplier*((mov + offset)**pow)/(a + b*(w_elo-l_elo))
  #k = 3.511

  e_w = 1/(1 + 10**((l_elo-w_elo)/400))
  e_l = 1/(1 + 10**((w_elo-l_elo)/400))

  #revert HCA away
  if df_games.loc[i, 'WLoc'] == 'H':
    w_elo = w_elo-HCA
  if df_games.loc[i, 'WLoc'] == 'A':
    l_elo = l_elo-HCA

  w_change = k*(1-e_w)
  w_elo = w_elo + w_change

  l_change = k*(-e_l)
  l_elo = l_elo + l_change

  elo[team1-3101] = w_elo + 0
  elo[team2-3101] = l_elo + 0

##Read in tournament seeds from file

In [ ]:
df_seeds = pd.read_csv(file_path + 'WNCAATourneySeeds.csv')
seeds = []
for year in range(start_year, end_year+1):
  seeds.append({})

for row in df_seeds.itertuples():
  year = row[1]
  seed = int(''.join(c for c in row[2] if c.isdigit()))
  team = int(row[3])
  seeds[year-start_year].update({team : seed})

##Process game results by season and compile ELO ratings

In [ ]:
multiplier = 500
offset = 0.5
pow = 0.8
a = 116
b = 0.001
HCA = 4
average_elo = 1000
simplifier = 0.9

elo = []
for team in range(3101, max_team):
  elo.append(average_elo)

df_games = pd.read_csv(file_path + 'WRegularSeasonDetailedResults.csv')
df_tourney_games = pd.read_csv(file_path + 'WNCAATourneyCompactResults.csv')
current_season = 2003

for i in range(len(df_games.index)):
  year = df_games.loc[i, 'Season']
  #covid
  if year == 2020:
    continue

  #if new season
  if year > current_season:

    with open('elo_by_year.csv', 'a') as csv_file:
      csvwriter = csv.writer(csv_file)
      if current_season == 2003:
        csvwriter.writerow(['Season', 'Team', 'Elo'])
      team_id = 3101
      for rating in elo:
        csvwriter.writerow([current_season, team_id, rating])
        team_id = team_id+1

    #reset season statistics
    current_season = year

    for i in range(len(elo)):
      elo[i] = simplifier*elo[i] + (1-simplifier)*average_elo

  #otherwise, update Elos
  process_elos(df_games, elo, i)

#complete file by writing present year
with open('elo_by_year.csv', 'a') as csv_file:
  csvwriter = csv.writer(csv_file)
  team_id = 3101
  for rating in elo:
    csvwriter.writerow([2024, team_id, rating])
    team_id = team_id+1

#Setup R environment

##Enable rpy2 for using R in python

In [ ]:
%load_ext rpy2.ipython

##Set the R logger to only display errors (this eliminates the wall of warnings)

In [ ]:
from rpy2.rinterface_lib.callbacks import logger as rpy2_logger
import logging
rpy2_logger.setLevel(logging.ERROR)   # will display errors, but not warnings

##Install the `xgboost` and `lme4` R packages

In [ ]:
%%R
install.packages("xgboost")

In [ ]:
%%R
install.packages("lme4")

#Implement the 2018 Raddar solution, combining his model with ELO ratings

##The Raddar solution can be found here: https://github.com/fakyras/ncaa_women_2018/blob/master/win_ncaa.R

In [ ]:
%%R
library(dplyr)
library(xgboost)
library(lme4)

regresults <- read.csv("./gdrive/MyDrive/March Madness 2024/WRegularSeasonDetailedResults.csv")
results <- read.csv("./gdrive/MyDrive/March Madness 2024/WNCAATourneyDetailedResults.csv")
sub <- read.csv("WSampleSubmissionStage2.csv")
seeds <- read.csv("./gdrive/MyDrive/March Madness 2024/WNCAATourneySeeds.csv")
elo <- read.csv('elo_by_year.csv')

seeds$Seed = as.numeric(substring(seeds$Seed,2,4))


### Collect regular season results - double the data by swapping team positions

r1 = regresults[, c("Season", "DayNum", "WTeamID", "WScore", "LTeamID", "LScore", "NumOT", "WFGA", "WAst", "WBlk", "LFGA", "LAst", "LBlk")]
r2 = regresults[, c("Season", "DayNum", "LTeamID", "LScore", "WTeamID", "WScore", "NumOT", "LFGA", "LAst", "LBlk", "WFGA", "WAst", "WBlk")]
names(r1) = c("Season", "DayNum", "T1", "T1_Points", "T2", "T2_Points", "NumOT", "T1_fga", "T1_ast", "T1_blk", "T2_fga", "T2_ast", "T2_blk")
names(r2) = c("Season", "DayNum", "T1", "T1_Points", "T2", "T2_Points", "NumOT", "T1_fga", "T1_ast", "T1_blk", "T2_fga", "T2_ast", "T2_blk")
regular_season = rbind(r1, r2)


### Collect tourney results - double the data by swapping team positions

t1 = results[, c("Season", "DayNum", "WTeamID", "LTeamID", "WScore", "LScore")] %>% mutate(ResultDiff = WScore - LScore)
t2 = results[, c("Season", "DayNum", "LTeamID", "WTeamID", "LScore", "WScore")] %>% mutate(ResultDiff = LScore - WScore)
names(t1) = c("Season", "DayNum", "T1", "T2", "T1_Points", "T2_Points", "ResultDiff")
names(t2) = c("Season", "DayNum", "T1", "T2", "T1_Points", "T2_Points", "ResultDiff")
tourney = rbind(t1, t2)


### Fit GLMM on regular season data (selected march madness teams only) - extract random effects for each team

march_teams = select(seeds, Season, Team = TeamID)
X =  regular_season %>%
  inner_join(march_teams, by = c("Season" = "Season", "T1" = "Team")) %>%
  inner_join(march_teams, by = c("Season" = "Season", "T2" = "Team")) %>%
  select(Season, T1, T2, T1_Points, T2_Points, NumOT) %>% distinct()
X$T1 = as.factor(X$T1)
X$T2 = as.factor(X$T2)

quality = list()
for (season in unique(X$Season)) {
  glmm = glmer(I(T1_Points > T2_Points) ~  (1 | T1) + (1 | T2), data = X[X$Season == season & X$NumOT == 0, ], family = binomial)
  random_effects = ranef(glmm)$T1
  quality[[season]] = data.frame(Season = season, Team_Id = as.numeric(row.names(random_effects)), quality = exp(random_effects[,"(Intercept)"]))
}
quality = do.call(rbind, quality)


### Regular season statistics

season_summary =
  regular_season %>%
  mutate(win14days = ifelse(DayNum > 118 & T1_Points > T2_Points, 1, 0),
         last14days = ifelse(DayNum > 118, 1, 0)) %>%
  group_by(Season, T1) %>%
  summarize(
    WinRatio14d = sum(win14days) / sum(last14days),
    PointsMean = mean(T1_Points),
    PointsMedian = median(T1_Points),
    PointsDiffMean = mean(T1_Points - T2_Points),
    FgaMean = mean(T1_fga),
    FgaMedian = median(T1_fga),
    FgaMin = min(T1_fga),
    FgaMax = max(T1_fga),
    AstMean = mean(T1_ast),
    BlkMean = mean(T1_blk),
    OppFgaMean = mean(T2_fga),
    OppFgaMin = min(T2_fga)
  )

season_summary <- season_summary %>%
left_join(select(elo, Season, Team, Elo), by = c("Season" = "Season", "T1" = "Team")) %>%
rename(Elo = Elo)

season_summary_X1 = season_summary
season_summary_X2 = season_summary
names(season_summary_X1) = c("Season", "T1", paste0("X1_",names(season_summary_X1)[-c(1,2)]))
names(season_summary_X2) = c("Season", "T2", paste0("X2_",names(season_summary_X2)[-c(1,2)]))


### Combine all features into a data frame

data_matrix =
  tourney %>%
  left_join(season_summary_X1, by = c("Season", "T1")) %>%
  left_join(season_summary_X2, by = c("Season", "T2")) %>%
  left_join(select(seeds, Season, T1 = TeamID, Seed1 = Seed), by = c("Season", "T1")) %>%
  left_join(select(seeds, Season, T2 = TeamID, Seed2 = Seed), by = c("Season", "T2")) %>%
  mutate(SeedDiff = Seed1 - Seed2) %>%
  left_join(select(quality, Season, T1 = Team_Id, quality_march_T1 = quality), by = c("Season", "T1")) %>%
  left_join(select(quality, Season, T2 = Team_Id, quality_march_T2 = quality), by = c("Season", "T2"))


### Prepare xgboost

features = setdiff(names(data_matrix), c("Season", "DayNum", "T1", "T2", "T1_Points", "T2_Points", "ResultDiff"))
dtrain = xgb.DMatrix(as.matrix(data_matrix[, features]), label = data_matrix$ResultDiff)

cauchyobj <- function(preds, dtrain) {
  labels <- getinfo(dtrain, "label")
  c <- 5000
  x <-  preds-labels
  grad <- x / (x^2/c^2+1)
  hess <- -c^2*(x^2-c^2)/(x^2+c^2)^2
  return(list(grad = grad, hess = hess))
}

xgb_parameters =
  list(objective = cauchyobj,
       eval_metric = "mae",
       booster = "gbtree",
       eta = 0.02,
       subsample = 0.35,
       colsample_bytree = 0.7,
       num_parallel_tree = 10,
       min_child_weight = 40,
       gamma = 10,
       max_depth = 3)

N = nrow(data_matrix)
fold5list = c(
  rep( 1, floor(N/5) ),
  rep( 2, floor(N/5) ),
  rep( 3, floor(N/5) ),
  rep( 4, floor(N/5) ),
  rep( 5, N - 4*floor(N/5) )
)


### Build cross-validation model, repeated 10-times

iteration_count = c()
smooth_model = list()

for (i in 1:10) {

  ### Resample fold split
  set.seed(i)
  folds = list()
  fold_list = sample(fold5list)
  for (k in 1:5) folds[[k]] = which(fold_list == k)

  set.seed(120)
  xgb_cv =
    xgb.cv(
      params = xgb_parameters,
      data = dtrain,
      nrounds = 3000,
      verbose = 0,
      nthread = 12,
      folds = folds,
      early_stopping_rounds = 25,
      maximize = FALSE,
      prediction = TRUE
    )
  iteration_count = c(iteration_count, xgb_cv$best_iteration)

  ### Fit a smoothed GAM model on predicted result point differential to get probabilities
  smooth_model[[i]] = smooth.spline(x = xgb_cv$pred, y = ifelse(data_matrix$ResultDiff > 0, 1, 0))

}


### Build submission models

submission_model = list()

for (i in 1:10) {
  set.seed(i)
  submission_model[[i]] =
    xgb.train(
      params = xgb_parameters,
      data = dtrain,
      nrounds = round(iteration_count[i]*1.05),
      verbose = 0,
      nthread = 12,
      maximize = FALSE,
      prediction = TRUE
    )
}


### Run predictions

sub$Season = 2024
sub$T1 = as.numeric(substring(sub$ID,6,9))
sub$T2 = as.numeric(substring(sub$ID,11,14))

Z = sub %>%
  left_join(season_summary_X1, by = c("Season", "T1")) %>%
  left_join(season_summary_X2, by = c("Season", "T2")) %>%
  left_join(select(seeds, Season, T1 = TeamID, Seed1 = Seed), by = c("Season", "T1")) %>%
  left_join(select(seeds, Season, T2 = TeamID, Seed2 = Seed), by = c("Season", "T2")) %>%
  mutate(SeedDiff = Seed1 - Seed2) %>%
  left_join(select(quality, Season, T1 = Team_Id, quality_march_T1 = quality), by = c("Season", "T1")) %>%
  left_join(select(quality, Season, T2 = Team_Id, quality_march_T2 = quality), by = c("Season", "T2"))

dtest = xgb.DMatrix(as.matrix(Z[, features]))

probs = list()
for (i in 1:10) {
  preds = predict(submission_model[[i]], dtest)
  probs[[i]] = predict(smooth_model[[i]], preds)$y
}
Z$Pred = Reduce("+", probs) / 10

write.csv(select(Z, ID, Pred), "sub.csv", row.names = FALSE)
